# **Python für Ingenieure**
<!-- Lizensiert unter (CC BY 4.0) Gert Herold, 2022 -->
# 6. NumPy

>NumPy is the fundamental package for scientific computing in Python. It is a Python library that provides a multidimensional array object, various derived objects (such as masked arrays and matrices), and an assortment of routines for fast operations on arrays, including mathematical, logical, shape manipulation, sorting, selecting, I/O, discrete Fourier transforms, basic linear algebra, basic statistical operations, random simulation and much more.
>
> &mdash; <cite>[NumPy.org](https://numpy.org/doc/stable/user/whatisnumpy.html), 2022</cite>

Das Python-Modul NumPy beinhaltet (mit seinen Untermodulen) eine mächtige Toolbox, die den Umgang mit Daten unter Python sehr komfortabel macht.

## 6.1. Arrays

Das wohl wichtigste Werkzeug, das NumPy mit sich bringt, ist der Datentyp [*ndarray*](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html), mit dem sich beliebig-dimensionale Vektoren/Matrizen/Tensoren (kurz: Arrays) verwirklichen lassen.

### 6.1.1. Arrays anlegen

Eine einfache Möglichkeit, ein Array zu erzeugen, ist mithilfe der [*array()*](https://docs.scipy.org/doc/numpy/reference/generated/numpy.array.html)-Funktion:

In [1]:
from numpy import array
import numpy as np
a = np.array([1, 2, 3, 6, 8])
print(a)
print(type(a))
print(a.dtype)


[1 2 3 6 8]
<class 'numpy.ndarray'>
int32


Im Unterschied zu einer Python-Liste sind Elemente eines Arrays für gewöhnlich numerisch sowie stets vom gleichen Typ. 
Eine Auflistung von möglichen Array-Datentypen findet sich in der [Numpy-Dokumentation](https://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html).
Befinden sich in einer als Array zu speichernden Liste Variablen unterschiedlichen Typs, wird, wenn möglich, ein Datentyp festgelegt, mit dem alle Werte abgebildet werden können.

In [2]:
liste_a = [1,   7, -3,  5, 12]
liste_b = [2, 7.5, 11, -1,  0]

a = array(liste_a)
b = array(liste_b)

print(f'Liste: {liste_a}\nArray: {a}\n{a.dtype}\n')
print(f'Liste: {liste_b}\nArray: {b}\n{b.dtype}\n')

Liste: [1, 7, -3, 5, 12]
Array: [ 1  7 -3  5 12]
int32

Liste: [2, 7.5, 11, -1, 0]
Array: [ 2.   7.5 11.  -1.   0. ]
float64



Mehrdimensionale Arrays lassen sich einfach über Unterlisten erzeugen:

In [3]:
c = array([[1,2],[3,4]])
print('array:\n',c)
print('dimension:',c.ndim)
print('shape:',c.shape)
print('size:',c.size)

array:
 [[1 2]
 [3 4]]
dimension: 2
shape: (2, 2)
size: 4


### 6.1.2. Indizierung von Arrays

Der Abruf von Elementen eines Arrays (*Slicing*) funktioniert ähnlich wie bei einer Liste bzw. die Möglichkeiten, eine Liste zu indizieren, funktionieren auch hier:

In [4]:
from numpy import array

liste_d = [[  1,  1,  3,  4,  5],
           [  1,  1,  8,  9, 10],
           [  1,  1, 13, 14, 15],
           [  1,  1, 18, 19, 20]]

d = array(liste_d)

print(liste_d[2][3])
print(d[2][3])

print(d[2])
print(d[0][::-1])

14
14
[ 1  1 13 14 15]
[5 4 3 1 1]


Darüber hinaus können die Indizierungen hier einfach per Komma abgetrennt (also als Tupel angegeben) werden:

In [5]:
d[2,3]

14

Das erlaubt auch ein einfaches Abrufen von Sub-Arrays:

In [6]:
e = d[:2,3:]
e

array([[ 4,  5],
       [ 9, 10]])

Diese Index-Operationen sind rechentechnisch sehr effizient, da hierdurch keine neuen Daten im Speicher hinterlegt werden, sondern die Einträge des neuen Arrays lediglich auf die bereits angelegten Daten zeigen.
Das hat zur Folge, dass Änderungen im neuen Array auch im ursprünglichen Array auftauchen:

In [7]:
e[0,1] = 12345
d

array([[    1,     1,     3,     4, 12345],
       [    1,     1,     8,     9,    10],
       [    1,     1,    13,    14,    15],
       [    1,     1,    18,    19,    20]])

Hilfreich ist auch die Möglichkeit, Integer-Arrays selbst zur Indizierung zu verwenden:

In [8]:
index = array([0,2,3])
d[:,index]

array([[ 1,  3,  4],
       [ 1,  8,  9],
       [ 1, 13, 14],
       [ 1, 18, 19]])

Es können auch Arrays mit Bool'schen Ausdrücken verwendet werden. Mit `False` können so Werte, die nicht gewünscht sind, maskiert werden:

In [9]:
b_ind = array([False, True, True, False, True])
b, b[b_ind]

(array([ 2. ,  7.5, 11. , -1. ,  0. ]), array([ 7.5, 11. ,  0. ]))

Bei Indizierung mit Integer- oder Boolean-Arrays werden 
  * direkt die Einträge angesprochen, bei denen der Wert `True` steht,
  * Kopien der Daten angelegt, wenn das Ergebnis der Indizierung einer weiteren Variable zugewiesen werden soll.

Für die meisten praktischen Anwendungen spielt es keine Rolle, ob die Daten kopiert oder ob nur Speicheradressen umgeleitet werden.
Bei der Fehlersuche in Programmen sollte man sich dessen jedoch bewusst sein.

### 6.1.3. Rechnen mit Arrays und Broadcasting

**Beispiel 1:** Es sollen die Elemente einer Liste mit den entsprechenden Elementen einer anderen Liste addiert werden. Dies ist z.B. möglich über eine Schleife:

In [10]:
neue_liste = []
for i,j in zip(liste_a, liste_b):
    neue_liste += [i+j]
print(liste_a)
print(liste_b)
print(neue_liste)

[1, 7, -3, 5, 12]
[2, 7.5, 11, -1, 0]
[3, 14.5, 8, 4, 12]


Etwas übersichtlicher wird das mit einer List Comprehension:

In [11]:
[i+j for i,j in zip(liste_a, liste_b)]

[3, 14.5, 8, 4, 12]

Liegen die Daten als Array vor, geht das noch einfacher:

In [12]:
a+b

array([ 3. , 14.5,  8. ,  4. , 12. ])

**Beispiel 2:** Wir wollen alle Werte einer Liste mit 100 multiplizieren. Auch hier muss diese Multiplikation für jedes Element einzeln vorgenommen werden:

In [13]:
[i*100 for i in liste_b]

[200, 750.0, 1100, -100, 0]

Rechenoperationen mit Arrays hingegen sind sehr effizient gestaltet.
Hier schreibt man einfach:

In [14]:
b*100

array([ 200.,  750., 1100., -100.,    0.])

Das funktioniert mit beliebig-dimensionalen Arrays:

In [15]:
d,d*100

(array([[    1,     1,     3,     4, 12345],
        [    1,     1,     8,     9,    10],
        [    1,     1,    13,    14,    15],
        [    1,     1,    18,    19,    20]]),
 array([[    100,     100,     300,     400, 1234500],
        [    100,     100,     800,     900,    1000],
        [    100,     100,    1300,    1400,    1500],
        [    100,     100,    1800,    1900,    2000]]))

Allgemein erlaubt NumPy die Kombination von zwei Arrays verschiedener Größen, wenn für die jeweils *hinteren Dimensionen* der Arrays eine der folgenden Bedingungen erfüllt ist:
  * die Anzahl der Elemente ist jeweils gleich
  * ein Array hat in dieser Dimension genau einen Eintrag
  
Im ersten Fall wird jedes Element eines Arrays mit dem entsprechenden Element des zweiten Arrays kombiniert. 
Im zweiten Fall wird das einzelne Element auf alle anderen angewendet.
Unterscheidet sich die Anzahl der Dimensionen (auch: Achsen bzw. engl. *axes*), so werden dem Array mit weniger Dimensionen entsprechend viele Achsen hinzugefügt.

Diese Funktionalitäten werden als [Broadcasting](https://numpy.org/devdocs/user/theory.broadcasting.html) bezeichnet.
Das bedeutet z.B. für das 2D-Array `d` mit den Dimensionen (4, 5)

In [16]:
d.shape

(4, 5)

dass Operation neben Skalaren z.B. auch mit Arrays mit den Dimensionen 
  * (4, 5)
  * (1, 1)
  * (1, 5)
  * (4, 1)
  * (5, )
  * (3, 4, 5)
  * (75, 1, 5)
  * usw.

möglich sind.

In [17]:
print(a.shape, d.shape,'\n')
print(a,'\n')
print(d,'\n')
print(a+d,'\n')

(5,) (4, 5) 

[ 1  7 -3  5 12] 

[[    1     1     3     4 12345]
 [    1     1     8     9    10]
 [    1     1    13    14    15]
 [    1     1    18    19    20]] 

[[    2     8     0     9 12357]
 [    2     8     5    14    22]
 [    2     8    10    19    27]
 [    2     8    15    24    32]] 



In [18]:
f = array([[10],[20],[30],[40]])
print(f.shape, d.shape,'\n')
print(f,'\n')
print(d,'\n')
print(f*d,'\n')

(4, 1) (4, 5) 

[[10]
 [20]
 [30]
 [40]] 

[[    1     1     3     4 12345]
 [    1     1     8     9    10]
 [    1     1    13    14    15]
 [    1     1    18    19    20]] 

[[    10     10     30     40 123450]
 [    20     20    160    180    200]
 [    30     30    390    420    450]
 [    40     40    720    760    800]] 



Falls die beiden Arrays nicht den obigen Regeln entsprechend kombinierbar sind, wird eine Fehlermeldung ausgegeben:

In [19]:
e+d

ValueError: operands could not be broadcast together with shapes (2,2) (4,5) 

Mitunter ist es aber gewünscht, mit allen Einzeleinträgen eines beliebigen Arrays Operationen auf ein Array anderer Dimension auszuführen. Nehmen wir zum Beispiel die obige Multiplikation des Arrays `d` mit der Zahl 100. Nun wollen wir dieses Array sowohl mit 100 als auch mit 200 multiplizieren:

In [20]:
print(d*100)
print(d*200)

# Geht das auch in einem Schritt?
mult = array([100,200])
print(d * mult)

[[    100     100     300     400 1234500]
 [    100     100     800     900    1000]
 [    100     100    1300    1400    1500]
 [    100     100    1800    1900    2000]]
[[    200     200     600     800 2469000]
 [    200     200    1600    1800    2000]
 [    200     200    2600    2800    3000]
 [    200     200    3600    3800    4000]]


ValueError: operands could not be broadcast together with shapes (4,5) (2,) 

Das ist so nicht möglich, weil entgegen den Broadcastregeln versucht wird, zwei Elemente fünf Elemente zu projizieren.
Das eigentliche Ziel war aber ein ganz anderes, nämlich *alle* Elemente des einen Arrays mit *allen* Elementen des anderen zu multiplizieren.
Hierfür müssen dem 1D-Array weitere Dimensionen/Achsen hinzugefügt werden.
Das geht mit dem Pseudo-Index `newaxis`:

In [21]:
from numpy import newaxis
print(d * mult[:, newaxis, newaxis])
print('\nShapes:', d.shape, (mult[:, newaxis, newaxis]).shape)

[[[    100     100     300     400 1234500]
  [    100     100     800     900    1000]
  [    100     100    1300    1400    1500]
  [    100     100    1800    1900    2000]]

 [[    200     200     600     800 2469000]
  [    200     200    1600    1800    2000]
  [    200     200    2600    2800    3000]
  [    200     200    3600    3800    4000]]]

Shapes: (4, 5) (2, 1, 1)


## Übung


In [ ]:
from numpy import array

x  =  [[ 0.332,  0.49 ,  0.743,  0.688,  0.914,  0.661],
       [ 0.004,  0.285,  0.334,  2.086, -0.164,  0.52 ],
       [ 0.118,  0.482,  0.64 , -1.468,  0.923,  0.088],
       [ 0.995,  0.946, -1.386, -0.75 ,  0.879, -1.105],
       [ 1.072, -0.628, -0.482,  0.838, -0.517, -0.094],
       [-0.125,  0.342,  0.858, -0.626, -2.145,  1.429],
       [ 1.832, -0.471,  0.085,  0.457,  1.449, -0.486]]

y  =  [[ 0.401, -0.468,  0.692, -0.505, -2.672, -0.132],
       [-0.296,  0.54 ,  0.712,  0.496, -0.615, -0.22 ],
       [-0.525, -0.208, -2.41 ,  0.346, -1.93 ,  0.42 ],
       [-0.786,  0.176, -0.652,  0.515,  1.069, -0.457],
       [-0.63 , -1.372, -0.483,  0.608,  1.295, -0.637],
       [-0.286,  1.546, -1.219, -1.779,  0.955, -0.424],
       [-0.435, -1.886, -0.34 , -1.293, -0.646, -1.366]]

ax = array(x)
ay = array(y)

**1) Addieren Sie die beiden Listen `x` und `y` elementweise (also `x[0][0]+y[0][0]`, `x[0][1]+y[0][1]`, usw). Vergleichen Sie die Rechenzeit, wenn Sie Listen verwenden, mit der Zeit, die Sie für die Berechnung mit den Arrays `ax` und `ay` benötigen.**

In [ ]:
%%timeit
# Hier eigenen Code für Listenoperation schreiben ...
output_ =list([i+j for i,j in zip(x,y)])

In [ ]:
ax+ay

In [ ]:
%%timeit
# Hier eigenen Code für Arrayoperation schreiben ...
ax+ay


**2) Geben Sie jede zweite *Zeile* des Arrays `ax` aus. Geben Sie jede zweite *Spalte* des Arrays `ay` aus..**

In [ ]:
# Hier eigenen Code schreiben ...
print(ax[::2])
print(ay[:,::2])



**3) Kopieren Sie die Werte der beiden Arrays, die *nicht* am Rand liegen, in zwei neue Arrays `xx` und `yy`.**

In [ ]:
x  =  [[ 0.332,  0.49 ,  0.743,  0.688,  0.914,  0.661],
       [ 0.004,  0.285,  0.334,  2.086, -0.164,  0.52 ],
       [ 0.118,  0.482,  0.64 , -1.468,  0.923,  0.088],
       [ 0.995,  0.946, -1.386, -0.75 ,  0.879, -1.105],
       [ 1.072, -0.628, -0.482,  0.838, -0.517, -0.094],
       [-0.125,  0.342,  0.858, -0.626, -2.145,  1.429],
       [ 1.832, -0.471,  0.085,  0.457,  1.449, -0.486]]

y  =  [[ 0.401, -0.468,  0.692, -0.505, -2.672, -0.132],
       [-0.296,  0.54 ,  0.712,  0.496, -0.615, -0.22 ],
       [-0.525, -0.208, -2.41 ,  0.346, -1.93 ,  0.42 ],
       [-0.786,  0.176, -0.652,  0.515,  1.069, -0.457],
       [-0.63 , -1.372, -0.483,  0.608,  1.295, -0.637],
       [-0.286,  1.546, -1.219, -1.779,  0.955, -0.424],
       [-0.435, -1.886, -0.34 , -1.293, -0.646, -1.366]]

ax = array(x)
ay = array(y)
# Hier eigenen Code schreiben ...
xx = ax[1:len(ax)-1, 1:len(ax[0])-1]
yy = ay[1:len(ax)-1, 1:len(ax[0])-1]
xx, yy

**4) Überschreiben Sie im Array `xx` die Werte, an deren Position im Array `yy` ein negativer Wert steht, mit dem Quadrat dieses Werts.**

In [ ]:
# Hier eigenen Code schreiben ...

# is_neg = yy < 0
# is_neg = np.where(is_neg == True, 2, is_neg)
# is_neg = np.where(is_neg == False, 1, is_neg)
# # print(is_neg)
x_pos = xx.copy()

print(x_pos)
# print(is_neg)
# x_pos = x_pos**is_neg

# print(x_pos)
# x_pos
print(yy<0)
x_pos[yy <0 ] = x_pos[yy<0]**2
print(x_pos)

**5) Was passiert im folgenden Programmabschnitt und wie erklärt sich das Ergebnis?**

In [ ]:
y_neu = ay[::2, 1::2]
print(y_neu)
y_neu[:] = 100
print(y_neu)
print(ay)